# Assignment 3

Welcome to Assignment 3. This will be even more fun. Now we will calculate statistical measures on the test data you have created.

YOU ARE NOT ALLOWED TO USE ANY OTHER 3RD PARTY LIBRARIES LIKE PANDAS. PLEASE ONLY MODIFY CONTENT INSIDE THE FUNCTION SKELETONS
Please read why: https://www.coursera.org/learn/exploring-visualizing-iot-data/discussions/weeks/3/threads/skjCbNgeEeapeQ5W6suLkA
. Just make sure you hit the play button on each cell from top to down. There are seven functions you have to implement. Please also make sure than on each change on a function you hit the play button again on the corresponding cell to make it available to the rest of this notebook.
Please also make sure to only implement the function bodies and DON'T add any additional code outside functions since this might confuse the autograder.

So the function below is used to make it easy for you to create a data frame from a cloudant data frame using the so called "DataSource" which is some sort of a plugin which allows ApacheSpark to use different data sources.


In [1]:
#Please don't modify this function
def readDataFrameFromCloudant(database):
    cloudantdata=spark.read.load(database, "org.apache.bahir.cloudant")

    cloudantdata.createOrReplaceTempView("washing")
    spark.sql("SELECT * from washing").show()
    return cloudantdata

All functions can be implemented using DataFrames, ApacheSparkSQL or RDDs. We are only interested in the result. You are given the reference to the data frame in the "df" parameter and in case you want to use SQL just use the "spark" parameter which is a reference to the global SparkSession object. Finally if you want to use RDDs just use "df.rdd" for obtaining a reference to the underlying RDD object. 

Let's start with the first function. Please calculate the minimal temperature for the test data set you have created. We've provided a little skeleton for you in case you want to use SQL. You can use this skeleton for all subsequent functions. Everything can be implemented using SQL only if you like.

In [2]:
def minTemperature(df,spark):
    return spark.sql("SELECT temperature as mintemp from washing where temperature  IS NOT NULL order by 1 limit 1").first().mintemp

Please now do the same for the mean of the temperature

In [3]:
def meanTemperature(df,spark):
    return spark.sql("SELECT AVG(temperature) as average  from washing  ").first().average

Please now do the same for the maximum of the temperature

In [4]:
def maxTemperature(df,spark):
    return spark.sql("SELECT temperature as mintemp from washing where temperature  IS NOT NULL order by 1 desc").first().mintemp

Please now do the same for the standard deviation of the temperature

In [5]:
def sdTemperature(df,spark):
    return spark.sql("SELECT STDDEV(temperature) as average  from washing  ").first().average

Please now do the same for the skew of the temperature. Since the SQL statement for this is a bit more complicated we've provided a skeleton for you. You have to insert custom code at four position in order to make the function work. Alternatively you can also remove everything and implement if on your own. Note that we are making use of two previously defined functions, so please make sure they are correct. Also note that we are making use of python's string formatting capabilitis where the results of the two function calls to "meanTemperature" and "sdTemperature" are inserted at the "%s" symbols in the SQL string.

In [6]:
def skewTemperature(df,spark):    
    n=spark.sql("SELECT count(temperature) as s  from washing  ").first().s
    return spark.sql("""
SELECT 
    (
        1/(%s)
    ) *
    SUM (
        POWER(temperature-%s,3)/POWER(%s,3)
    )

as zstd from washing
                    """ %(n,meanTemperature(df,spark),sdTemperature(df,spark))).first().zstd

Kurtosis is the 4th statistical moment, so if you are smart you can make use of the code for skew which is the 3rd statistical moment. Actually only two things are different.

In [7]:
def kurtosisTemperature(df,spark):    
    n=spark.sql("SELECT count(temperature) as s  from washing  ").first().s
    return spark.sql("""
SELECT 
    (
        1/(%s)
    ) *
    SUM (
        POWER(temperature-%s,4)/POWER(%s,4)
    )

as zstd from washing
                    """ %(n,meanTemperature(df,spark),sdTemperature(df,spark))).first().zstd

Just a hint. This can be solved easily using SQL as well, but as shown in the lecture also using RDDs.

In [8]:
from pyspark.mllib.stat import Statistics
def correlationTemperatureHardness(df,spark):
    return Statistics.corr(sc.parallelize(spark.sql("SELECT temperature,hardness as x FROM washing where temperature IS NOT NULL and hardness IS NOT NULL").collect()).map(lambda row: [row[0],row[1]]))

### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED
#axx
### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED

Now it is time to connect to the cloudant database. Please have a look at the Video "Overview of end-to-end scenario" of Week 2 starting from 6:40 in order to learn how to obtain the credentials for the database. Please paste this credentials as strings into the below code

### TODO Please provide your Cloudant credentials here

In [9]:
hostname = "d9ce7ec2-6e11-49eb-98cf-ebbb93eaea72-bluemix.cloudant.com"
user = "d9ce7ec2-6e11-49eb-98cf-ebbb93eaea72-bluemix"
pw = "4980828b2d3aa7f2fbab39d158c1902580926a9d9f8947cc8d1951c79151aa55"
database = "washing" #as long as you didn't change this in the NodeRED flow the database name stays the same

In [10]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",hostname)\
    .config("cloudant.username", user)\
    .config("cloudant.password",pw)\
    .getOrCreate()
cloudantdata=readDataFrameFromCloudant(database)

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0cacb9bbe89157ca9...|1-5e9d57927c4a54c...|    1|      11|acceptable|     null|      73| null|         92|1540071828400|   null|
|0cacb9bbe89157ca9...|1-9940fa7baf9b4d9...|    8|      11|acceptable|     null|      77| null|         92|1540071835736|   null|
|0cacb9bbe89157ca9...|1-84a327f5ffd5098...|   11|      11|acceptable|     null|      70| null|         83|1540071838743|   null|
|0cacb9bbe89157ca9...|1-527212aae71495c...|   14|      11|acceptable|     null|      70| null|         82|1540071841876|   null|
|21aae38c942c13765...|1-b31ff9fb152eeda...|    3|      11|acceptable|     null|      72| null|   

In [11]:
minTemperature(cloudantdata,spark)

80

In [12]:
meanTemperature(cloudantdata,spark)

88.42857142857143

In [13]:
maxTemperature(cloudantdata,spark)

99

In [14]:
sdTemperature(cloudantdata,spark)

6.443075935702412

In [15]:
skewTemperature(cloudantdata,spark)

0.2334941524635015

In [16]:
kurtosisTemperature(cloudantdata,spark)

1.529664965097152

In [17]:
correlationTemperatureHardness(cloudantdata,spark)

array([[ 1.        ,  0.32179104],
       [ 0.32179104,  1.        ]])

Congratulations, you are done, please download this notebook as python file using the export function and submit is to the gader using the filename "assignment3.1.py"